In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\jurge\AppData\Local\Temp\ipykernel_20524\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [4]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [5]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

## Data Preprocessing

In [6]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [7]:
def clean_text(text):  
    text = re.sub(r"<script.*?>.*?</script>", "", text, flags=re.DOTALL)
    text = re.sub(r"<style.*?>.*?</style>", "", text, flags=re.DOTALL)

    text = re.sub(r"<!--.*?-->", "", text, flags=re.DOTALL)
    
    text = re.sub(r"<.*?>", "", text)

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [11]:
import re

def clean_text(text): 
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    text = re.sub(r"\s+[a-zA-Z]\s+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    text = text.lower()
    return text

## Now let's work on removing stopwords
Remove the stopwords.

In [16]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

def remove_stopwords(text):
    words = text.split()
    return " ".join([word for word in words if word not in stop_words])

    data['preprocessed_text'] = data['text']

data["preprocessed_text"] = data["preprocessed_text"].apply(remove_stopwords)

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [18]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
nltk.download('punkt_tab')

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = word_tokenize(text)
    return " ".join([lemmatizer.lemmatize(word) for word in words])

data["preprocessed_text"] = data["preprocessed_text"].apply(lemmatize_text)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\jurge\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
word_counts = vectorizer.fit_transform(data["preprocessed_text"])

word_counts_df = pd.DataFrame(word_counts.toarray(), columns=vectorizer.get_feature_names_out())

top_words = word_counts_df.sum().sort_values(ascending=False).head(10)
print("Top 10 Words:")
print(top_words)


Top 10 Words:
2e         1857
the        1550
br         1103
you         989
money       985
to          931
2c          924
account     876
of          875
bank        804
dtype: int64


## Extra features

In [24]:
# Money symbols and suspicious words
money_symbols = "|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = "|".join(["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"])

data["money_mark"] = data["preprocessed_text"].str.contains(money_symbols).astype(int)
data["suspicious_words"] = data["preprocessed_text"].str.contains(suspicious_words).astype(int)
data["text_len"] = data["preprocessed_text"].apply(len)

data.head()


,text,label,preprocessed_text,money_mark,suspicious_words,text_len
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,"DEAR SIR , STRICTLY A PRIVATE BUSINESS PROPOSA...",1,0,2325
1,Will do.,0,Will do .,1,0,9
2,Nora--Cheryl has emailed dozens of memos about...,0,Nora -- Cheryl emailed dozen memo Haiti weeken...,1,0,127
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,Dear Sir=2FMadam=2C I know proposal might surp...,1,1,1554
4,fyi,0,fyi,1,0,3


## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [25]:
tfidf_vectorizer = TfidfVectorizer()

tfidf_data = tfidf_vectorizer.fit_transform(data["preprocessed_text"])

print(f"TF-IDF Dataset Shape: {tfidf_data.shape}")

TF-IDF Dataset Shape: (1000, 25156)


## And the Train a Classifier?

In [26]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

X_train, X_val, y_train, y_val = train_test_split(
    tfidf_data, data["label"], test_size=0.2, random_state=42
)
clf = MultinomialNB()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_val)

print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95       125
           1       0.86      1.00      0.93        75

    accuracy                           0.94       200
   macro avg       0.93      0.95      0.94       200
weighted avg       0.95      0.94      0.94       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code